In [ ]:
from btrdb_v4_api import btrdb_v4_api
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import json

# Python API Demo

In [ ]:
conn = btrdb_v4_api()
conn.info()

#### Create A Demo Dataset

In [ ]:
endTime = datetime.now()
startTime = endTime - timedelta(hours=10)
data = pd.DataFrame({}, index=pd.date_range(startTime, endTime, freq='S'))
data['A_el_Power_W'] = np.where((data.index.minute<9) | (data.index.minute>58), 0, 35) # unit is [W] Watt
data['B_th_Window1_C'] = 20 + np.sin(data.index.astype(int)/1.5e13-3) * 5 # unit is [C] Celsius
data

#### Create A New Stream
Here we created two new streams "A_el_Power_W" and "B_th_Window1_C".

In [ ]:
conn.create_stream("A_el_Power_W")
conn.create_stream("B_th_Window1_C")

#### Lookup Streams
We can list the stream names given the collection. 

In [ ]:
display(conn.list_streams('71t/A', True, True))
display(conn.list_streams('71t/B', True, True))

In [ ]:
conn.list_streams('71t/B', True, True)[0]['stream_name'].decode("utf-8") 

Or we can list the stream objects given the collection

In [ ]:
display(conn.list_streams('71t/A', True, False))
display(conn.list_streams('71t/B', True, False))

#### Retrieve Real Stream UUID
We can retrieve the UUID of "A_el_Power_W" and "B_th_Window1_C"

In [ ]:
display(conn.retrieve_uuid("A_el_Power_W"))
display(conn.retrieve_uuid("B_th_Window1_C"))

#### Insert Data 
We can insert Data into the streams

In [ ]:
conn.wirte_data(data.iloc[:100000, :])

#### Retrieve Data 
We then read the data we just inserted in min window.

In [ ]:
conn.read_window_data(startTime, endTime, 'h', ['A_el_Power_W', 'B_th_Window1_C'])

We can also read data in raw value points.

In [ ]:
conn.read_raw_data(startTime, endTime, ['A_el_Power_W', 'B_th_Window1_C'])

#### Delete Data Range

In [ ]:
conn.delete_range(startTime, endTime,['A_el_Power_W','B_th_Window1_C'])

#### Delete Streams
We can delete streams given the stream keynames/labels. For example, we are deleting "A_el_Power_W" and "B_th_Window1_C" here.

In [ ]:
conn.delete_stream(["A_el_Power_W", "B_th_Window1_C"])

In [ ]:
display(conn.list_streams('71t/A', True, True))
display(conn.list_streams('71t/B', True, True))

# Http API Tutorial

Import libraries

In [ ]:
from btrdb_v4_api import btrdb_v4_api
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import json

Generate Some Demo Data

In [ ]:
endTime = datetime.now()
startTime = endTime - timedelta(hours=10)
data = pd.DataFrame({}, index=pd.date_range(startTime, endTime, freq='S'))
data['A_el_Power_W'] = np.where((data.index.minute<9) | (data.index.minute>58), 0, 35) # unit is [W] Watt
data['B_th_Window1_C'] = 20 + np.sin(data.index.astype(int)/1.5e13-3) * 5 # unit is [C] Celsius
data

Check connection

In [ ]:
response = requests.get("https://71tdb.lbl.gov:9090/info")
print(response.json())

Signup

In [ ]:
payload = {'username':'demo', 'email':'demo@gmail.com', 'password': 'demo123'}
response = requests.post("https://71tdb.lbl.gov:9090/signup", json=json.dumps(payload))
response.text

Login

In [ ]:
payload = {'username':'demo', 'password': 'demo123'}
response = requests.post("https://71tdb.lbl.gov:9090/login", json=json.dumps(payload))
#It is important to save the cookies after login
cookies = response.cookies
response.text

Logout

In [ ]:
response = requests.post("https://71tdb.lbl.gov:9090/logout", cookies=cookies)
response.text

Write (streams will be automatically created if it doesn't exist already)

In [ ]:
payload = data.iloc[:100, :].to_json()
response = requests.put("https://71tdb.lbl.gov:9090/write_db", json=payload, cookies=cookies)
if response.status_code != 200:
    print(response.text)

Get metadata/uuid

In [ ]:
response = requests.get("https://71tdb.lbl.gov:9090/get_metadata/A_el_Power_W", cookies=cookies)
response.json()

Read Window Data

In [ ]:
payload = {'start': startTime, 
           'end':endTime,
           'ts': 'min',
           'keynames': ['A_el_Power_W','B_th_Window1_C']
          }
response = requests.get("https://71tdb.lbl.gov:9090/read_window_db", params=payload, cookies=cookies)
response.json()

Read Raw Points

In [ ]:
payload = {'start': startTime, 
           'end':endTime,
           'keynames': ['A_el_Power_W','B_th_Window1_C']
          }
response = requests.get("https://71tdb.lbl.gov:9090/read_raw_db", params=payload, cookies=cookies)
response.json()

Delete Range Data

In [ ]:
payload = {'start': startTime, 
           'end':endTime,
           'keynames': ['A_el_Power_W','B_th_Window1_C']
          }
response = requests.get("https://71tdb.lbl.gov:9090/delete_range_db", params=payload, cookies=cookies)
response.json()

Delete Streams

In [ ]:
payload = {'keynames': ['A_el_Power_W','B_th_Window1_C']}
response = requests.get("https://71tdb.lbl.gov:9090/delete_streams", params=payload, cookies=cookies)
response.json()

Watchdog Add Stream

In [ ]:
payload = {'A_el_Power_W': {'stucktime':30, 'missingtime':None}, 
           'B_th_Window1_C': {'stucktime':30, 'missingtime':None}}
payload = json.dumps(payload)
response = requests.put("https://71tdb.lbl.gov:9090/watchdog/add_stream", json=payload, cookies=cookies)

Watchdog Delete Streams

In [ ]:
payload = {'streams': ['A_el_Power_W', 'B_th_Window1_C']}
payload = json.dumps(payload)
response = requests.put("https://71tdb.lbl.gov:9090/watchdog/delete_stream", json=payload, cookies=cookies)

Watchdog Add Rooms

In [ ]:
payload = {'rooms': ['A','B']}
payload = json.dumps(payload)
response = requests.put("https://71tdb.lbl.gov:9090/watchdog/add_room", json=payload, cookies=cookies)

Watchdog Delete Rooms

In [ ]:
payload = {'rooms': ['A','B']}
payload = json.dumps(payload)
response = requests.put("https://71tdb.lbl.gov:9090/watchdog/delete_room", json=payload, cookies=cookies)

# Client


Import Libraries

In [ ]:
from btrdb_v4_api import btrdb_v4_api
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import json
from client import client

Generate Some Demo Data

In [ ]:
endTime = datetime.now()
startTime = endTime - timedelta(hours=3)
data = pd.DataFrame({}, index=pd.date_range(startTime, endTime, freq='S'))
data['A_el_Power_W'] = np.where((data.index.minute<9) | (data.index.minute>58), 0, 35) # unit is [W] Watt
data['B_th_Window1_C'] = 20 + np.sin(data.index.astype(int)/1.5e13-3) * 5 # unit is [C] Celsius
data

In [ ]:
c = client()

Signup

In [ ]:
c.signup()

Login

In [ ]:
c.login()

Login with one step (note this is not secure for passwords, since we are saving the clear text pasword in the script here)

In [ ]:
c.login_no_interactive('oliver', '1234')

Logout

In [ ]:
c.logout()

Get Connection Info

In [ ]:
c.info()

Get Metadata

In [ ]:
c.get_metadata('A_el_Power_W')

Write Data

In [ ]:
c.write_db(data.iloc[:10000, :])

Read Window Data

In [ ]:
c.read_window_db(startTime, endTime, 's', ['A_el_Power_W','B_th_Window1_C'])

Read Raw Data

In [ ]:
c.read_raw_db(startTime, endTime, ['A_el_Power_W','B_th_Window1_C'])

Delete Range Data

In [ ]:
c.delete_range_data(startTime , endTime, ['A_el_Power_W','B_th_Window1_C'])

Delete Streams

In [ ]:
c.delete_streams(['A_el_Power_W','B_th_Window1_C'])

Watchdog Add Streams

In [ ]:
payload = {'A_el_Power_W': {'stucktime':30, 'missingtime':None}, 
           'B_th_Window1_C': {'stucktime':30, 'missingtime':None}}
payload = json.dumps(payload)
c.watchdog_addStream(payload)

Watchdog Delete Streams

In [ ]:
payload = {'streams': ['A_el_Power_W', 'B_th_Window1_C']}
payload = json.dumps(payload)
print(c.watchdog_deleteStream(payload))


Watchdog Add Rooms

In [ ]:
payload = {'rooms': ['A','B']}
payload = json.dumps(payload)
print(c.watchdog_addRoom(payload))

Watchdog Delete Rooms

In [ ]:
payload = {'rooms': ['A','B']}
payload = json.dumps(payload)
print(c.watchdog_deleteRoom(payload))